In [ ]:
import pandas
import plotly.offline as offline
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
"""Read data
Info about data: https://www.kaggle.com/IHME/us-countylevel-mortality
Mortality = deaths per 100,000 people, combined for both sexes, age standardized"""
d = pandas.read_csv("mort.csv.gz")
"""Display header"""
d.head()

In [ ]:
"""Print column names"""
d.columns

In [ ]:
"""How many rows and columns do we have?"""
d.shape

In [ ]:
"""Retain only useful columns and display header"""
d1 = d[['Location', 'FIPS', 'Category', 'Mortality Rate, 1980*', 'Mortality Rate, 2014*', '% Change in Mortality Rate, 1980-2014']]
d1.head()

In [ ]:
"""Question: Which disease causes the highest mortality rates in the USA for 2014?"""
"""Make dataframe to only contain mortality values for the whole USA, for all disease categories"""
dus = d1[d1['Location']=="United States"]

In [ ]:
"""Sort datafame by mortality rate and plot a horizontal bar plot"""
dus.sort_values(by=['Mortality Rate, 2014*']).plot(x='Category', 
                                                   y='Mortality Rate, 2014*', 
                                                   kind='barh', 
                                                   title="Mortality rate in 2014", 
                                                   legend=False, 
                                                   figsize=(10,4))
plt.tight_layout()
plt.savefig("mortality2014.pdf")

In [ ]:
"""Since our dataset contained mortality rates over time, we can ask: how did mortality rates change from the year 1980 to 2014?"""
"""Plot %change in mortality from 1980-2014 in USA for all diseases"""
dus.sort_values(by=['% Change in Mortality Rate, 1980-2014']).plot(x='Category', 
                                                                   y='% Change in Mortality Rate, 1980-2014', 
                                                                   kind='barh', 
                                                                   title="% change in mortality from 1980 to 2014", 
                                                                   legend=False,
                                                                  figsize=(10,4))
plt.tight_layout()
plt.savefig("rateOfChange.png")
plt.show()

In [ ]:
"""Having done analyses at the country level, move into the county level. Make new dataframe to conatin only county information.
Also subset to only contain information about Cardiovascular diseases."""
dstate = d1[d1['FIPS'] < 100]
dcounty = d1[d1['FIPS'] > 100]
dstate.loc[:,'FIPS'] = dstate['FIPS'].astype(int)
dcounty.loc[:,'FIPS'] = dcounty['FIPS'].astype(int)
dcounty = dcounty[dcounty['Category'] == "Cardiovascular diseases"]

In [ ]:
"""Since our dataset already has FIPS values for all counties, they can be plotted on the USA map.
Use plotly.figure_factory function create_choropleth to plot USA heatmap
Color by Cardiovascular mortality rate in 2014"""

cols = ['rgb(255,255,255)',
 'rgb(49,54,149)',
 'rgb(69,117,180)',
 'rgb(116,173,209)',
 'rgb(244,109,67)',
 'rgb(215,48,39)',
 'rgb(165,0,38)',
 'rgb(0,0,0)']

m2014 = ff.create_choropleth(fips=dcounty['FIPS'].tolist(), 
                             values=dcounty['Mortality Rate, 2014*'].tolist(), 
                             title="Cardiovascular Disease Mortality Rates in 2014",
                             binning_endpoints=[0, 100, 200, 300, 400, 500, 600],
                             colorscale=cols,
                             legend_title="Deaths per 100,000 people")

offline.plot(m2014, filename="fig.CVD_2014.html")

In [ ]:
"""Zoom into Michigan!"""

dmi = dcounty[dcounty['Location'].str.contains("Michigan")]
m2014_MI = ff.create_choropleth(fips=dmi['FIPS'].tolist(),
                              values=dmi['Mortality Rate, 2014*'].tolist(), 
                              title="Cardiovascular Disease Mortality Rates 2014", 
                              scope=['MI','IL','IN','OH','WI'], 
                              legend_title="Deaths per 100,000 people", 
                              binning_endpoints=[150, 200, 250, 300, 350, 400], 
                              colorscale=cols)

offline.plot(m2014_MI, filename="michigan2014_1.html")

In [ ]:
"""Found another dataset showing education information by USA counties. 
Hypothesis: Cardiovascular mortality rates negatively correlate with higher education
Read education data and select relevant columns, display header"""

edu = pandas.read_excel("Education.xls", header=4, usecols=['FIPS Code', 'State', 'Area name','Percent of adults with less than a high school diploma, 2012-2016',
       'Percent of adults with a high school diploma only, 2012-2016',
       "Percent of adults completing some college or associate's degree, 2012-2016",
       "Percent of adults with a bachelor's degree or higher, 2012-2016"])
edu.head()

In [ ]:
"""Merge the mortality dataset with the education dataset"""
dm = pandas.merge(dcounty, edu, how="inner", left_on="FIPS", right_on="FIPS Code")
dm.head()

In [ ]:
"""Check if county names don't match after merging by FIPS"""
dm[['county','state']]=dm['Location'].str.split(',',expand=True)
dm[dm['county'] != dm['Area name']]

In [ ]:
sns.jointplot(data=dm, x='Mortality Rate, 2014*', y="Percent of adults with a bachelor's degree or higher, 2012-2016", kind="reg")

In [ ]:
sns.jointplot(data=dm, x='Mortality Rate, 2014*', y='Percent of adults with less than a high school diploma, 2012-2016', kind="reg")

In [ ]:
dstate = dstate[dstate['Category'] == "Cardiovascular diseases"]
dstate.loc[:,'FIPS'] = dstate['FIPS'].astype(int) * 1000
ds = pandas.merge(dstate, edu, how="inner", left_on="FIPS", right_on="FIPS Code")

In [ ]:
sns.jointplot(data=ds, x='Mortality Rate, 2014*', y='Percent of adults with less than a high school diploma, 2012-2016', kind="reg")

In [ ]:
sns.jointplot(data=ds, x='Mortality Rate, 2014*', y="Percent of adults with a bachelor's degree or higher, 2012-2016", kind="reg")

In [ ]:
sns.jointplot(data=dm, x='Mortality Rate, 2014*', y='Percent of adults with less than a high school diploma, 2012-2016', kind="hex")

In [ ]:
medu = ff.create_choropleth(fips=dm['FIPS'].tolist(), 
                             values=dm['Percent of adults with less than a high school diploma, 2012-2016'].tolist(), 
                             title="Percent of adults with less than a high school diploma, 2012-2016",
                             binning_endpoints=[0,10,20,30,40,50],
                           colorscale=cols)

offline.plot(medu, filename="fig.percentAdults_lessThanHighSchool.html")